In [89]:
import pandas as pd
import numpy as np
from collections import defaultdict

def load_data():
    disputes = pd.read_csv('data/MIDB_4.01.csv')
    country_codes = pd.read_csv('data/COW country codes.csv').drop_duplicates()
    states_by_year = pd.read_csv('data/system2011.csv') # State membership in each year of dataset

    # Get rid of columns we don't care about
    disputes = disputes.loc[:, ['StAbb', 'ccode', 'StYear', 'EndYear', 'Orig', 'Fatality', 'FataPre', 'HiAct']]
    states_by_year = states_by_year.loc[:, ['stateabb', 'ccode', 'year']]
    
    # Only include hostile acts that involved at least a border violation
    disputes = disputes[disputes['HiAct'] >= 12]
    
    # Filter out countries who were the aggressor (since we are studying deterrence)
    disputes = disputes[disputes['Orig'] == 0]
    
    return disputes, states_by_year

disputes, states_by_year = load_data()

In [66]:
def get_membership_set(states_by_year):
    years = states_by_year['year'].unique()

    # Make a dict to track state membership by year
    membership = defaultdict(set)

    # Add states that existed each year to dict
    for year in years:
        membership[year].update(states_by_year[states_by_year['year'] == year]['ccode'])
    
    return membership
        
membership = get_membership_set(states_by_year)

In [83]:
def get_total_possible_conflicts(disputes):
    
    # Add the countries that existed in each year but did not have a conflict
    new_data = list()
    for year in range(1816, 2011):

        # Find the countries that existed AND had a conflict for given year
        filter_criteria = disputes['StYear'].map(lambda x: x == year)

        # Find countries that existed but were not attacked in given year
        were_attacked = set(disputes[disputes['StYear'] == year]['ccode'])
        not_attacked = membership[year] - were_attacked

        for country in not_attacked:
            # Add to original data frame (conflict-related columns should be NaN)
            new_data.append({'ccode': country, 'were_attacked': 0, 'StYear': year, 'EndYear': year})

    # Append the data (once, from dict for better performance)
    total_possible_conflicts = disputes.append(pd.DataFrame.from_dict(new_data), ignore_index=True)

    # Correct NaNs for countries that were attacked
    total_possible_conflicts.update(total_possible_conflicts['were_attacked'].fillna(1))
    
    return total_possible_conflicts

total_possible_conflicts = get_total_possible_conflicts(disputes)

In [84]:
def add_data(conflicts):
    
    # Add materiel capabilities
    nmc = pd.read_csv('data/NMC_v4_0.csv')
    conflicts = pd.merge(left=conflicts, right=nmc, left_on=['ccode', 'StYear'], 
                                        right_on=['ccode', 'year'])

    # Merge with country code names to get the full state name
    conflicts = pd.merge(left=conflicts, right=country_codes, 
                         left_on='ccode', right_on='CCode', how='inner')
    
    return conflicts
    
total_possible_conflicts = add_data(total_possible_conflicts)

In [120]:
def get_interwar_durations(total_possible_conflicts):
    """Adds the duration between being attacked, as a possible explanatory variable.  For left-censored
    data, uses the median duration between attacked for fully-observed periods."""

    # Sort conflict data by country then year
    total_possible_conflicts = total_possible_conflicts.sort_values(by=['CCode', 'StYear'], ascending=[True, True])
    # Get list of all countries in dataset to track when they are no longer left-censored
    censored_countries = pd.Series(True, index=states_by_year['ccode'].unique())
    # Track duration of periods currently observing
    current_periods = pd.Series(0, index=states_by_year['ccode'].unique())
    # Track results
    fully_observed_periods = list() # For calculating the median for left-censored data
    total_possible_conflicts['years_since_attacked'] = pd.Series(np.nan, index=total_possible_conflicts.index)

    # Get all fully-observed durations since last conflict in data set. 
    # Iterate by year since some years have multiple entries (if a state was involved in a multi-party dispute 
    # during a single year).
    for year in range(1816, 2011):
        year_data = total_possible_conflicts[total_possible_conflicts['StYear'] == year]
        for ccode in states_by_year['ccode'].unique():
            country_data = year_data[year_data['ccode'] == ccode]

            # Find instances of given country in given year being attacked
            conflicts_for_year = country_data.loc[(total_possible_conflicts['were_attacked'] == 1)]

            # Update years since attacked for the given country in given year
            updated = country_data['years_since_attacked'].fillna(current_periods[ccode])
            total_possible_conflicts['years_since_attacked'].update(updated)
            
            # If the country was attacked
            if(len(conflicts_for_year.index) >= 1):
                if censored_countries[ccode] == False:
                    fully_observed_periods.append(current_periods[ccode])
                # Reset period
                current_periods[ccode] = 0
                # Make sure country isn't censored
                censored_countries[ccode] = False
            
            # Increment the period
            current_periods[ccode] += 1
    
    # Anything left as NaN must be left-censored data; update this with the median duration between conflicts
    total_possible_conflicts['years_since_attacked'].fillna(pd.Series(fully_observed_periods).median())
    
    return total_possible_conflicts

total_possible_conflicts = get_interwar_durations(total_possible_conflicts)

In [127]:
total_possible_conflicts[(total_possible_conflicts['ccode'] == 2) & (total_possible_conflicts['year'] >= 1915)]

EndYear  FataPre  Fatality  HiAct  Orig StAbb  StYear  ccode  \
2249     1915      NaN       NaN    NaN   NaN   NaN    1915      2   
2250     1916      NaN       NaN    NaN   NaN   NaN    1916      2   
2138     1918       -9         6     20     0   USA    1917      2   
2136     1920       -9        -9     17     0   USA    1918      2   
2137     1920       -9        -9     14     0   USA    1918      2   
2251     1919      NaN       NaN    NaN   NaN   NaN    1919      2   
2252     1920      NaN       NaN    NaN   NaN   NaN    1920      2   
2253     1921      NaN       NaN    NaN   NaN   NaN    1921      2   
2254     1922      NaN       NaN    NaN   NaN   NaN    1922      2   
2255     1923      NaN       NaN    NaN   NaN   NaN    1923      2   
2256     1924      NaN       NaN    NaN   NaN   NaN    1924      2   
2257     1925      NaN       NaN    NaN   NaN   NaN    1925      2   
2258     1926      NaN       NaN    NaN   NaN   NaN    1926      2   
2134     1927       -9        -9     16     0   USA    1927      2   
2259     1928      NaN       NaN    NaN   NaN   NaN    1928      2   
2142     1929        0         0     16     0   USA    1929      2   
2260     1930      NaN       NaN    NaN   NaN   NaN    1930      2   
2261     1931      NaN       NaN    NaN   NaN   NaN    1931      2   
2262     1932      NaN       NaN    NaN   NaN   NaN    1932      2   
2263     1933      NaN       NaN    NaN   NaN   NaN    1933      2   
2264     1934      NaN       NaN    NaN   NaN   NaN    1934      2   
2265     1935      NaN       NaN    NaN   NaN   NaN    1935      2   
2266     1936      NaN       NaN    NaN   NaN   NaN    1936      2   
2267     1937      NaN       NaN    NaN   NaN   NaN    1937      2   
2268     1938      NaN       NaN    NaN   NaN   NaN    1938      2   
2269     1939      NaN       NaN    NaN   NaN   NaN    1939      2   
2270     1940      NaN       NaN    NaN   NaN   NaN    1940      2   
2139     1945       -9         6     20     0   USA    1941      2   
2140     1941        0         0     18     0   USA    1941      2   
2271     1942      NaN       NaN    NaN   NaN   NaN    1942      2   
...       ...      ...       ...    ...   ...   ...     ...    ...   
2302     1978      NaN       NaN    NaN   NaN   NaN    1978      2   
2303     1979      NaN       NaN    NaN   NaN   NaN    1979      2   
2304     1980      NaN       NaN    NaN   NaN   NaN    1980      2   
2305     1981      NaN       NaN    NaN   NaN   NaN    1981      2   
2306     1982      NaN       NaN    NaN   NaN   NaN    1982      2   
2307     1983      NaN       NaN    NaN   NaN   NaN    1983      2   
2308     1984      NaN       NaN    NaN   NaN   NaN    1984      2   
2309     1985      NaN       NaN    NaN   NaN   NaN    1985      2   
2310     1986      NaN       NaN    NaN   NaN   NaN    1986      2   
2311     1987      NaN       NaN    NaN   NaN   NaN    1987      2   
2145     1988        0         0     16     0   USA    1988      2   
2312     1989      NaN       NaN    NaN   NaN   NaN    1989      2   
2146     1991       -9         3     20     0   USA    1990      2   
2313     1991      NaN       NaN    NaN   NaN   NaN    1991      2   
2314     1992      NaN       NaN    NaN   NaN   NaN    1992      2   
2315     1993      NaN       NaN    NaN   NaN   NaN    1993      2   
2316     1994      NaN       NaN    NaN   NaN   NaN    1994      2   
2317     1995      NaN       NaN    NaN   NaN   NaN    1995      2   
2318     1996      NaN       NaN    NaN   NaN   NaN    1996      2   
2319     1997      NaN       NaN    NaN   NaN   NaN    1997      2   
2147     1999        0         0     16     0   USA    1998      2   
2320     1999      NaN       NaN    NaN   NaN   NaN    1999      2   
2321     2000      NaN       NaN    NaN   NaN   NaN    2000      2   
2322     2001      NaN       NaN    NaN   NaN   NaN    2001      2   
2149     2005        7         1     17     0   USA    2002      2   
2323     2003   

In [113]:
import statsmodels.api as sm

# Ensure no NaN values in training data (some countries are missing from NMC data set)
total_possible_conflicts = total_possible_conflicts.dropna(subset=['milex', 'milper', 'pec', 'tpop', 'upop'])

# http://blog.yhat.com/posts/logistic-regression-and-python.html
training_data = total_possible_conflicts[['milex', 'milper', 'pec', 'tpop', 'upop', 'years_since_attacked']]
logit = sm.Logit(total_possible_conflicts['were_attacked'], training_data, missing='raise')
result = logit.fit()
result.summary()

# To do make it so years since attacked is NaN while war is ongoing (because other parties enter disputes frequently
# when a war is ongoing, making it appear as though duration is more important than it really is). If they are NaN,
# will be dropped prior to the analysis

Optimization terminated successfully.
         Current function value: 0.137245
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          were_attacked   No. Observations:                14187
Model:                          Logit   Df Residuals:                    14181
Method:                           MLE   Df Model:                            5
Date:                Sun, 05 Mar 2017   Pseudo R-squ.:                 -0.4188
Time:                        17:04:24   Log-Likelihood:                -1947.1
converged:                       True   LL-Null:                       -1372.4
                                        LLR p-value:                     1.000
========================================================================================
                           coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
milex                -1.607e-09    3.9e-09     -0.412      0.680     -9.25e-09  6.04e-09
milper                   0.0002   7.64e-05      2.151      0.031      1.46e-05     0.000
pec                    1.06e-06   3.48e-07      3.047      0.002      3.78e-07  1.74e-06
tpop                 -2.756e-05   3.13e-06     -8.801      0.000     -3.37e-05 -2.14e-05
upop                   4.21e-05   7.21e-06      5.840      0.000       2.8e-05  5.62e-05
years_since_attacked    -0.0343      0.001    -44.770      0.000        -0.036    -0.033
========================================================================================
"""